In [1]:
import pandas as pd
import numpy as np

In [2]:
#problema con las propiedades leyendolo directo del excel -> por eso lo pasamos a csv
df = pd.read_csv('/Users/luisalawson/Downloads/Sample Data/epl.csv', delimiter=';')

In [3]:
df_prop = pd.DataFrame({
    'column_name': df.columns,
    'data_type': df.dtypes.values
})

In [4]:
df_vals = pd.DataFrame({
    'column_name': df.columns,
    'column_values': [set(df[col].tolist()) for col in df.columns],
    'unique_value_count': [len(set(df[col].tolist())) for col in df.columns]
})

In [5]:
def get_team_id_matches(team, df):
    idvalues = df[(df['home_team_name'] == team) | (df['away_team_name'] == team)]['match_id'].unique()
    return idvalues

In [6]:
def get_lineups(team_id, df):
    df_team = df[df['team_id'] == team_id] 
    lineups = set()
    for index, row in df_team.iterrows():
        lineups.add(row['playerName'])
        
    return lineups

In [7]:
def build_team_info(df):
    team_info = {}

    # to organize the info for a better access

    # we will use the team name as keys
    team_names = set(df['home_team_name']).union(df['away_team_name'])
    team_ids = set(df['home_team_id']).union(df['away_team_id'])

    # fill our dict
    for team_name in team_names:
        
        # gather team id data
        if (df['home_team_name'] == team_name).any():
            team_ids_for_name = df[df['home_team_name'] == team_name]['home_team_id'].unique()
        elif (df['away_team_name'] == team_name).any():
            team_ids_for_name = df[df['away_team_name'] == team_name]['away_team_id'].unique()
        
        team_id = team_ids_for_name[0] if len(team_ids_for_name) > 0 else None
        
        team_info[team_name] = {
            'team_id': team_id,
            'matches': get_team_id_matches(team_name, df),
            'lineups': get_lineups(team_id, df)
        }

    return team_info

In [15]:
def game_outcome(df):
    match_id_values = df['match_id'].unique()
    match_outcomes = []
    shot_categories = ['Attempt Saved', 'Miss', 'Goal', 'Out']
    defensive_events_1 = ['Tackle', 'Interception', 'Clearance', 'Blocked Pass', 'Ball Recovery'] 
    defensive_events_0 = ['Tackle', 'Failed to Block'] 
    
    for match in match_id_values:
        
        df_match = df[df['match_id'] == match]
        
        # should be the same teams on every row
        home_team = df_match['home_team_name'].iloc[0]
        away_team = df_match['away_team_name'].iloc[0]

        # useful to know which player scored (to add goal to that team) 
        home_team_id = df_match['home_team_id'].iloc[0]
        away_team_id = df_match['away_team_id'].iloc[0]

        # same as home_team
        date = df_match['date'].iloc[0]
        
        # goal calculation
        home_score = df_match[(df_match['team_id'] == home_team_id) & (df_match['description'] == "Goal")]['outcome'].sum()
        away_score = df_match[(df_match['team_id'] == away_team_id) & (df_match['description'] == "Goal")]['outcome'].sum()

        #shot calculation
        # attempt saved in description 
        # miss
        # goal (?
        # out(?
        home_shots = ((df_match['team_id'] == home_team_id) & (df_match['description'].isin(shot_categories))).sum()
        away_shots = ((df_match['team_id'] == away_team_id) & (df_match['description'].isin(shot_categories))).sum()

        #possession calculation
        home_possession = (df_match['team_id'] == home_team_id).sum()
        away_possession = (df_match['team_id'] == away_team_id).sum()

        # Defense calculation

        total_defenses_home = ((df_match['team_id'] == home_team_id) & ((df_match['description'].isin(defensive_events_0)) | (df_match['description'].isin(defensive_events_1)))).sum()
        total_defenses_away = ((df_match['team_id'] == away_team_id) & ((df_match['description'].isin(defensive_events_0)) | (df_match['description'].isin(defensive_events_1)))).sum()

        succesfull_defense_home = ((df_match['team_id'] == home_team_id) & ( (  (df_match['description'].isin(defensive_events_0)) & (df_match['outcome'] == 0) ) | (  (df_match['description'].isin(defensive_events_1)) & (df_match['outcome'] == 1) ))).sum()
        succesfull_defense_away = ((df_match['team_id'] == away_team_id) & ( (  (df_match['description'].isin(defensive_events_0)) & (df_match['outcome'] == 0) ) | (  (df_match['description'].isin(defensive_events_1)) & (df_match['outcome'] == 1) ))).sum()

        #PSL calculation
        home_PSL = home_shots/home_possession
        away_PSL = away_shots/away_possession

        #PSL diff
        home_PSL_dif = home_PSL - away_PSL
        away_PSL_dif = away_PSL - home_PSL

        # Calculo DC
        dc_home = succesfull_defense_home / total_defenses_home
        dc_away = succesfull_defense_away / total_defenses_away

        dc_dif = abs(dc_home- dc_away)

        dc_accurate = 0
    
        if(home_score > away_score and dc_home > dc_away):
            dc_accurate += 1
        elif(home_score < away_score and dc_home < dc_away):
            dc_accurate += 1
        elif(home_score == away_score and dc_dif < 0.1):
            dc_accurate += 1
        else:
            dc_accurate += 0

        # recover all match
        match_outcomes.append({
            'match_id': match,
            'date': date,
            'home_team': home_team,
            'away_team': away_team,
            'home_score': home_score,
            'away_score': away_score,
            'home_shots':home_shots,
            'away_shots':away_shots,
            'home_possesion': home_possession,
            'away_possession': away_possession,
            'home_PSL':home_PSL,
            'away_PSL': away_PSL,
            'home_PSL_dif': home_PSL_dif,
            'away_PSL_dif': away_PSL_dif,
            'dc_home': dc_home,
            'dc_away': dc_away,
            'dc_dif': dc_dif,
            'dc_accuracy': dc_accurate
        })
    
    
    outcome_df = pd.DataFrame(match_outcomes)
    
    return outcome_df  

In [16]:
match_results_df = game_outcome(df)
# not accurate --> we found og (own goals) are not distinguished from sg (successful goals)
# eg. 19May2013	Wigan Athletic- Aston Villa	--> 1- 3 was actually 2-2 for an aston villa player scored an og in the minute 45

match_results_df

,match_id,date,home_team,away_team,home_score,away_score,home_shots,away_shots,home_possesion,away_possession,home_PSL,away_PSL,home_PSL_dif,away_PSL_dif,dc_home,dc_away,dc_dif,dc_accuracy
0,441999,18Aug2012,Arsenal,Sunderland,0,0,78,59,1104,684,0.070652,0.086257,-0.015605,0.015605,0.977778,0.862069,0.115709,0
1,442000,20Aug2012,Everton,Manchester United,1,0,79,77,731,1081,0.108071,0.071230,0.036841,-0.036841,0.756410,0.771930,0.015520,0
2,442001,18Aug2012,Fulham,Norwich City,5,0,65,59,918,676,0.070806,0.087278,-0.016472,0.016472,0.727273,0.796296,0.069024,0
3,442002,19Aug2012,Manchester City,Southampton,3,2,78,66,1051,741,0.074215,0.089069,-0.014854,0.014854,0.811321,0.760000,0.051321,1
4,442003,18Aug2012,Newcastle United,Tottenham Hotspur,2,1,65,71,795,769,0.081761,0.092328,-0.010567,0.010567,0.709677,0.682540,0.027138,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,442374,19May2013,Swansea City,Fulham,0,3,71,52,1056,778,0.067235,0.066838,0.000397,-0.000397,0.979592,0.703704,0.275888,0
376,442375,19May2013,Tottenham Hotspur,Sunderland,1,0,110,83,956,678,0.115063,0.122419,-0.007356,0.007356,0.852941,0.737500,0.115441,1
377,442376,19May2013,West Bromwich Albion,Manchester United,6,4,74,69,659,955,0.112291,0.072251,0.040040,-0.040040,0.804878,0.862745,0.057867,0
378,442377,19May2013,West Ham United,Reading,4,2,98,95,782,767,0.125320,0.123859,0.001461,-0.001461,0.818182,0.872727,0.054545,0


In [17]:
excel_file_path = '/Users/luisalawson/Downloads/match_results.xlsx'  

match_results_df.to_excel(excel_file_path, index=False)

print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /Users/luisalawson/Downloads/match_results.xlsx


In [18]:
match_results_df['dc_accuracy'].sum()

np.int64(196)

In [21]:
match_results_df['dc_away'].mean()

np.float64(0.7806896718714951)